In [ ]:
import os
import librosa
import numpy
import tensorflow as tf
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
import Paths

directory = os.listdir(Paths.IN_DIRECTORY)
array = []

sequence_length = []
for sample_name in directory:
    if 'wav' in sample_name:
        file_path = os.path.join(Paths.IN_DIRECTORY, sample_name, sample_name)
        song, sr = librosa.load(file_path)
        sequence_length.append(len(song))
        array.append(song)


# Removing the last element using slicing
lastElementIndex = len(array)-1
in_array = array[:lastElementIndex]
out_array = array[1:]


#Create the Input and Output to tensors
input_tensor = tf.convert_to_tensor(in_array)
output_tensor = tf.convert_to_tensor(out_array)


batch_size = len(input_tensor)
timestep = sequence_length[0]

input_data = tf.reshape(input_tensor, shape=(batch_size, timestep, 1))
output_data = tf.reshape(output_tensor, shape=(batch_size, timestep, 1))

# Test mit Conv1D Layer, MaxPooling1D Layer, Aktivierungsfunktion Sigmoid
# Mögliche Anpassungen: Anzahl der Samples, Aktivierungsfunktion

# define model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3,strides=1, padding="causal", activation="sigmoid",input_shape=[None, 1]))
model.add(MaxPooling1D(pool_size=2, strides=1, padding='same'))
model.add(LSTM(100, activation='sigmoid', return_sequences=True, input_shape=(timestep, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()

# fit model
model.fit(input_data, output_data, epochs=25, verbose=2)
